In [2]:
import pyhf
import numpy as np
import matplotlib.pyplot as plt

In [5]:
model = pyhf.simplemodels.hepdata_like([6], [9], [np.sqrt(9)])

signal_pars = model.config.suggested_init()
signal_pars[model.config.poi_index] = 1.0

bkg_pars = model.config.suggested_init()
bkg_pars[model.config.poi_index] = 0.0

signal_pdf = model.make_pdf(signal_pars)
bkg_pdf = model.make_pdf(bkg_pars)

signal_sample = signal_pdf.sample((20000,))
bkg_sample = bkg_pdf.sample((20000,))

In [ ]:
signal_tests = np.asarray([pyhf.utils.hypotest(1.0, sample, model) for sample in signal_sample])
bkg_tests = np.asarray([pyhf.utils.hypotest(1.0, sample, model) for sample in bkg_sample])

In [ ]:
_, b, _ = plt.hist(signal_tests, bins=np.linspace(0, 10, 26), histtype="step", density=True)
_, b, _ = plt.hist(bkg_tests, bins=np.linspace(0, 10, 26), histtype="step", density=True)
plt.xlim(0, 9)
plt.ylim(1e-3, 2)
plt.semilogy()
plt.gcf().set_size_inches(5,5)